# Naive Bayes

Naive Bayes is a classification algorithm based on Bayes' theorem. Bayes’ theorem provides a way to calculate the probability of a data point belonging to a given class, given our prior knowledge. It is defined as

$$
P (class|data) = \frac{P (data|class) \ P (class)}{P (data)} ,
$$

where $P (class | data)$ is the probability over the potential classes given the provided data. The different probabilities $P$ you see in the equations above are commonly called prior, likelihood, evidence, and posterior as follows.

$$
\overbrace{P (class|data)}^{\text{posterior}} = \frac{\overbrace{P (data|class)}^{\text{likelihood}} \ \overbrace{P (class)}^{\text{prior}}}{\underbrace{P (data)}_{\text{evidence}}}
$$

The algorithm is called 'naive', because of its assumption that features of data are independent given the class label. Let us call the data features $x_1, \dots, x_j, \dots, x_p$ and the class label $y$, and rewrite Bayes theorem in these terms:

$$
P (y|x_1, \dots, x_p) = \frac{P (x_1, \dots, x_p|y) * P (y)}{P (x_1, \dots, x_p)} \, .
$$

Then, the naive assumption of conditional independence between any two features given the class label can be expressed as

$$
P(x_1, \dots, x_p | y) = \prod_{j=1}^p P(x_j | y) \, ,
$$

and now Bayes' theorem leads to:

$$
P (y | x_1, \dots, x_p) = \frac{P (y) \prod_{j=1}^p P(x_j | y)}{P (x_1, \dots, x_p)} \, .\tag{1}
$$

<!-- Since $P (x_1, \dots, x_p)$ is the constant input, we can define the following proportional relationship

$$
P (y|x_1, \dots, x_p) \propto P (y) \prod_{i=1}^p P(x_i | y) \, ,
$$

and can use it to classify any data point as

$$
\hat y = \underset{y}{\text{arg max}} \ P (y) \prod_{i=1}^p P(x_i | y) \, .
$$ -->

We can use the posterior distribution to classify any data point as

$$
\hat y = \underset{y}{\text{arg max}} \ P (y| x_1, \dots, x_p)\, .\tag{2}
$$

## Prepare data for text classification

To learn how this algorithm works in practice, we define a simple data set of emails being either spam or not (adopted from Chapter 3.5, Exercise 3.22 in Machine Learning: A Probabilistic Perspective by Murphy). _Note that Naive Bayes can indeed be used for multiclass classification (see lecture notes), however we use it here as a binary classifier._

We will work with the packages numpy and pandas.

In [200]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from typing import Callable

# Imports used for testing.
import numpy.testing as npt

SMALL_SIZE = 12
MEDIUM_SIZE = 16
BIGGER_SIZE = 20
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

Following Murphy, we create a toy dataset for spam email classification.

In [201]:
# define vocabulary
vocab = [
        'secret', 'offer', 'low', 'price', 'valued', 'customer', 'today',
        'dollar', 'million', 'sports', 'is', 'for', 'play', 'healthy', 'pizza'
    ]

# define train spam emails
spam = [
    'million dollar offer',
    'secret offer today',
    'secret is secret'
]

# define train non-spam emails
not_spam = [
    'low price for valued customer',
    'play secret sports today',
    'sports is healthy',
    'low price pizza'
]

Next we need to bring the toy data into a numerical form fit for applying machine learning models. We define the $N\times p$ [*document-term matrix*](https://en.wikipedia.org/wiki/Document-term_matrix) $X$, which counts the frequency of $p$ different words in a corpus of $N$ documents. The document-term matrix is a standard data format in natural language processing (NLP) related to the so-called *bag-of-words model*. The rows of $X$ denoted $\mathbf{x}^{(i)}, i=1,\ldots,N$, correspond to the documents and are represented by vectors $\mathbf{x}^{(i)}=(x^{(i)}_1,\ldots,x^{(i)}_p)$ where $x^{(i)}_j$ encodes the frequency of word $j$ in document $i$. Moreover, $y^{(i)}$ is the binary target variable that encodes whether document $i$ is spam ($y^{(i)}=1$) or not ($y^{(i)}=0$).

In [202]:
def prepare_spam_dataset(vocab, spam, not_spam, show_X=True):
    """ Prepare spam toy dataset for MultinomialNB implementation.

    Parameters:
        vocab (list[str]): List of words considered for document-term matrix
        spam (list[str]): List of spam documents
        not_spam (list[str]): List of non-spam documents
        show_X (bool): Prints document-term matrix as table


    Returns:
        X (np.array): Document-term matrix
        y (np.array): Indicator of whether or not message is spam
    """

    # corpus consists of spam and non-spam documents
    corpus = spam + not_spam

    # get size of corpus
    N = len(corpus)

    # get size of vocabulary
    p = len(vocab)

    # create dictionary for vocabulary indices
    vocab_dict = {vocab[j] : j for j in range(p)}

    # compute document term matrix
    X = np.zeros((N,p))

    # iterate through words
    for i, doc in enumerate(corpus):
        # split string into list of words
        doc_words = doc.split()
        # iterate through words and update document-term matrix
        for word in doc_words:
            if word in vocab:
                j = vocab.index(word)
                X[i,j] += 1
    if show_X:
        display(pd.DataFrame(X,columns=vocab))

    # encode class of each document
    y = [1] * len(spam) + [0] * len(not_spam)  # storing our labels in a list (1 means spam email, 0 means no spam email)

    return X, y

In [203]:
# define our variables and print document-term matrix X
X, y = prepare_spam_dataset(vocab, spam, not_spam, show_X=True)
print(y)

,secret,offer,low,price,valued,customer,today,dollar,million,sports,is,for,play,healthy,pizza
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
6,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


[1, 1, 1, 0, 0, 0, 0]


By looking at the document-term matrix $X$ one can already recognize some patterns in the relationship between words and spam. For example, the word secret appears three times in spam emails (the first three rows) but only one time in non-spam documents (the last four rows).

## Likelihood in Multinomial Naive Bayes

Next, we train the Naive Bayes classifier with a `mnb_fit()` function where we define the prior and likelihood.

Recall from our lectures that the prior is the probability distribution incorporating our knowledge of the classes (here spam and not-spam) and it can be directly computed from $y$ using the equation for the prior from the lecture notes.

Furthermore, we compute the likelihood of a word appearing in a document given its class (spam or not-spam) under the assumption of multinomiallly distributed data. In particular,  we estimate $P(x_j|y)$ as the relative frequency of term $j\in{1,2,...,p}$ in the documents of class $y$ and we also apply *Laplace smoothing* to avoid zero probabilities. This leads to

$$
P(x_j|y) = \frac{1+\sum_{i=1}^N x_j^{(i)}I(y^{(i)} = y )}{p + \sum_{i=1}^N \left(\sum_{j=1}^p x_j^{(i)}\right)I(y^{(i)} = y)}, \tag{3}
$$

which ensures that $P(x_1|y)+\ldots+P(x_p|y)=1$. This version of the Naive Bayes classifier is also called [*Multinomial Naive Bayes*](https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Multinomial_naive_Bayes) (MNB).


In [204]:
# EDIT THIS FUNCTION
def mnb_fit(X, y):
    """ Use training data to fit Multinomial Naive Bayes classifier.

    Parameters:
      X (np.array): Features
      y (np.array): Categorical target

    Returns:
      prior (np.array): Prior distribution of classes
      lk_word (np.array): Likelihood of words (features) to appear given class
    """

    # check if size of document-term matrix and target match
    assert X.shape[0] == len(y), "Size of document-term matrix and target does not match."

    # get size of vocabulary
    p = X.shape[1]

    # define prior
    prior =  np.array([np.mean(y==c) for c in np.unique(y)])

    # reorder X as a 2-dimensional array; each dimension contains data examples of only one of our two classes
    X_by_class = [X[y==c] for c in np.unique(y)]

    # count words in each class
    word_counts = np.array([X_c.sum(axis=0) for X_c in X_by_class])

    # define likelihood P(x|y) using Laplace smoothing, shape: (Nc, n)
    lk_word = (1 + word_counts) / (p + word_counts.sum(axis=1, keepdims=True))

    return prior, lk_word

**Verification:** To verify your implementation of `mnb_fit()`, you should expect the following cell to execute without error messages.

In [205]:
# The three lines shoud verify your implementation of mnb_fit()
# on a small example of two documents
prior_verify, lk_word_verify = mnb_fit(np.array([[0,1],[1,0]]), np.array([0,1]))
npt.assert_allclose(prior_verify, np.array([0.5,0.5]))
npt.assert_allclose(lk_word_verify, np.array([[1/3,2/3],[2/3,1/3]]))

We can now compute the prior and likelihood for our document-term matrix defined above.

In [206]:
# call function and print prior and likelihood.
prior, lk_word = mnb_fit(X, y)
print('Prior:', prior)
print('\n----------------')
print('Likelihood that word is typical for not_spam: \n', lk_word[0])
print('\n----------------')
print('Likelihood that word is typical for spam: \n', lk_word[1])

Prior: [0.57142857 0.42857143]

----------------
Likelihood that word is typical for not_spam: 
 [0.06666667 0.03333333 0.1        0.1        0.06666667 0.06666667
 0.06666667 0.03333333 0.03333333 0.1        0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]

----------------
Likelihood that word is typical for spam: 
 [0.16666667 0.125      0.04166667 0.04166667 0.04166667 0.04166667
 0.08333333 0.08333333 0.08333333 0.04166667 0.08333333 0.04166667
 0.04166667 0.04166667 0.04166667]


#### Questions:
1. What is the meaning of the likelihood $P(x_j|y)$ in the context of our toy example?
2. Plot the likelihood $P(x_j|y)$ of each word $j$ given the different classes and explain where the difference comes from.

## Posterior in Multinomial Naive Bayes

Now we can predict whether any given email is spam or not. To do so we compute the posterior probability $P(y|\mathbf{x})$ that a document $\mathbf{x}$ is part of class $y$, which, in the context of Multinomial Naive Bayes is given by:

$$
P(y|x_1,\ldots,x_p) \propto P(y) ∏_{j=1}^p P(x_j|y)^{x_j}\tag{4}
$$

As multiplication of many small values can lead to significant rounding errors, it's advantagous to carry out this computation in log space:

$$
\log P(y|x_1,\ldots,x_p) \propto \log P(y) + \sum_{j=1}^p x_j \, \log P(x_j|y)\tag{5}
$$

Note that the log-posterior is linear. We now implement a function that calculates the log-posterior using linear algebra and then returns the normalized (!) posterior probabilities.

In [207]:
# EDIT THIS FUNCTION
def mnb_predict_proba(X, prior, lk_word):
    """ Compute posterior probabiliy of class with Multinomal Naive Bayes.

    Params:
      X (np.array): Features
      prior (np.array): Prior distribution of classes
      lk_word (np.array): Likelihood of words (features) to appear given class

    Returns:
      posteriors (np.array): Posterior distribution of documents
    """

    # compute log-posterior
    log_posterior = np.log(prior) + X @ np.log(lk_word).T
    posterior = np.exp(log_posterior)
    # normalize to get full posterior distribution
    normalize_term = posterior.sum(axis=0, keepdims=True)
    posteriors = (posterior / normalize_term)

    return posteriors

**Verification:** To verify your implementation of `prdict_proba()`, you should expect the following cell to execute without error messages.

In [208]:
# The two lines shoud verify your implementation of mnb_predict_proba()
posteriors_verify = mnb_predict_proba(np.array([[0,1],[1,0]]), prior_verify, lk_word_verify)
npt.assert_allclose(posteriors_verify,np.array([[2/3,1/3],[1/3,2/3]]))

We can now compute the posterior probabilities for our training data.

In [209]:
# compute full posterior distribution
posteriors = mnb_predict_proba(X, prior, lk_word)
print("Posteriors:\n", posteriors)

Posteriors:
 [[2.27894257e-02 1.68220205e-01]
 [9.11577028e-02 3.36440410e-01]
 [1.82315406e-01 4.48587213e-01]
 [1.82315406e-03 2.43374139e-05]
 [1.82315406e-02 4.67278347e-03]
 [2.73473108e-01 2.80367008e-02]
 [4.10209663e-01 1.40183504e-02]]


Finally, we can classify the documents in a binary fashion by asserting any data points $X$ to the class $y$ with the highest probability (called the *argmax choice*).

In [210]:
# EDIT THIS FUNCTION
def predict(X, prior, lk_word):
    """ Predict class with highest probability.

    Params:
      X (np.array): Features
      prior (np.array): Prior distribution of classes
      lk_word (np.array): Likelihood of words (features) to appear given class

    Returns:
      y_pred (np.array): Predicted target
    """

    # prediction given by argmax choice
    predicted_probabilities = mnb_predict_proba(X, prior, lk_word)
    y_pred = np.argmax(predicted_probabilities, axis=1)

    return y_pred

Here, we take the same emails we used for training our Multinomial Naive Bayes classifier to also to evaluate it. Usually, the evaluation happens on unseen emails (test data) and it is your task below to define a small test dataset. What are the predicted classes and what is the accuracy of the classifier?

In [211]:
# predict targets for training data with Multinomial Naive Bayes
preds = predict(X, prior, lk_word)
print("Predicted classes: ", preds)
# compute accuracy
print(f'Train accuracy: {1 - np.linalg.norm((y - preds), ord=1) / len(preds)}') 

Predicted classes:  [1 1 1 0 0 0 0]
Train accuracy: 1.0


#### Questions:
1. Explain why the use of a multinomial distribution for the likelihood is justified when applying Naive Bayes to the document-term matrix.
2. At the example of Multinomial Naive Bayes for document classification, explain why Naive Bayes is called naive.
Can you think of a way to model text data differently so that Naive Bayes better captures the semantics of a document?



In [216]:
# Define test dataset
test_data = [
    'million dollar offer only today',
    'secret offer for customer',
    'low price offer for pizza',
    'valued secret for healthy pizza',
    'secrets of healthy sports'
]

def prepare_test_data(test_data, vocab):

    # get size of test data
    N = len(test_data)

    # get size of vocabulary
    p = len(vocab)
    vocab_dict = {vocab[j] : j for j in range(p)}

    # compute document term matrix
    X = np.zeros((N,p))

    # iterate through words
    for i, doc in enumerate(test_data):
        # split string into list of words
        doc_words = doc.split()
        # iterate through words and update document-term matrix
        for word in doc_words:
            if word in vocab:
                j = vocab.index(word)
                X[i,j] += 1
    return X

In [217]:
X_test = prepare_test_data(test_data, vocab)
y_test = [1,1,0,0,0]

preds = predict(X_test, prior, lk_word)
print(preds)
print("Predicted classes: ", preds)
# compute accuracy
print(f'Train accuracy: {1 - np.linalg.norm((y_test - preds), ord=1) / len(preds)}') 

[1 1 0 1 0]
Predicted classes:  [1 1 0 1 0]
Train accuracy: 0.8


#### Questions:
3. Define your own three short emails as a test set and evaluate our Naive Bayes classifier on it without re-training it on them. What do you observe?
4. What words have you included in emails of the test set that make them being classified as spam or not spam?

#### Questions:
5. Can you replicate your results using [sklearn](https://scikit-learn.org/stable/modules/naive_bayes.html)?

In [223]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X, y)
preds = mnb.predict(X_test)
print(preds)

[1 1 0 0 0]
